# UDS emulator 

## Preliminaries

In [1]:
from my_imports import *

In [2]:
yaml_path = 'config_file.yaml'
yaml_data = utils.load_yaml(yaml_path)

# Directories
training_simulations_path   = yaml_data['training_simulations_path']
validation_simulations_path = yaml_data['validation_simulations_path']

inp_path =  yaml_data['inp_path']

## Create simulation objects from data

In [3]:
list_of_training_simulations    = os.listdir(training_simulations_path)
list_of_validation_simulations  = os.listdir(validation_simulations_path)

print('Number of training simulations: ', len(list_of_training_simulations))
print('Number of validation simulations: ', len(list_of_validation_simulations))


Number of training simulations:  163
Number of validation simulations:  29


In [4]:
num_events = 5

validation_simulations = utils.extract_simulations_from_folders(validation_simulations_path, inp_path, max_events =num_events)

validation_windows = []
events_to_validate= list(range(num_events))
for event in events_to_validate:
    sim = validation_simulations[event]
    validation_windows += sim.get_all_windows(steps_ahead = 3)

The file does not have [STORAGE]

The file does not have '[PUMPS]\n'
The file does not have '[ORIFICES]\n'
The file does not have '[WEIRS]\n'


In [5]:
num_events = 5
training_simulations = utils.extract_simulations_from_folders(training_simulations_path, inp_path, max_events = num_events)

training_windows = []
events_to_train= list(range(num_events))
for event in events_to_train:
    sim = training_simulations[event]
    training_windows += sim.get_all_windows(steps_ahead = 3)

The file does not have [STORAGE]

The file does not have '[PUMPS]\n'
The file does not have '[ORIFICES]\n'
The file does not have '[WEIRS]\n'


In [6]:
normalizer = Normalizer(training_windows)

In [7]:
normalized_training_windows = normalizer.get_list_normalized_training_windows()
training_loader = DataLoader(normalized_training_windows, batch_size=32, shuffle = True)

In [8]:
normalized_validation_windows = [normalizer.normalize_window(val_win) for val_win in validation_windows]
validation_loader = DataLoader(normalized_validation_windows, batch_size=32)

In [9]:
gnn_model = GNNModel()
optimizer = optim.Adam(gnn_model.parameters())
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
loss_fn = nn.MSELoss()

In [ ]:
history = train(
    model = gnn_model,
    optimizer = optimizer,
    scheduler = scheduler,
    loss_fn = loss_fn,
    train_dl = training_loader,
    val_dl = validation_loader,
    epochs = 5,
    device = 'cpu',
    report_freq = 2
    )

: 

In [ ]:
pd.DataFrame(history)

: 

: 

#### Rolled-out validation event

: 

## Separate samples in training/validation/testing

: 

: 

## Data preprocessing

### Normalization

: 

### Visualization

## Model

### Artificial Neural Network definition

### Model definition

### Dynamic emulator

: 

: 

### Model training

### Model validation

### Model testing

## Predictions visualization

## Export model

# Scratchpad

In [ ]:
len(normalized_validation_windows)

351

: 